# Rerankers

In [1]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score) # -5.65234375

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
score = reranker.compute_score(['query', 'passage'], normalize=True)
print(score) # 0.003497010252573502

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores) # [-8.1875, 5.26171875]

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], normalize=True)
print(scores) # [0.00027803096387751553, 0.9948403768236574]


/Users/james/Documents/idat/genai-course/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[-5.6484375]
[0.003510649113774665]
[-8.1875, 5.26171875]
[0.00027803096387751553, 0.9948403768236574]


# Let's try with our db

In [15]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import os
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_openai import OpenAIEmbeddings

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")


client = QdrantClient(
    url=url,
    api_key=api_key,
    https=True,
    timeout=300
)

vector_store_page = QdrantVectorStore(
    client=client,
    collection_name="db-book-page",
    embedding=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

In [5]:
retriever = vector_store_page.as_retriever(search_kwargs={"k": 5})

In [6]:
query = "quién es susan fletcher?"

docs = retriever.invoke(input=query)

In [7]:
docs

[Document(metadata={'source': 'data/fortaleza-digital.pdf', 'page': 104, 'page_label': '105', '_id': '87bd3f3c-cced-4c43-8ec4-0aeb7dbcda78', '_collection_name': 'db-book-page'}, page_content='había tocado antes. No era el primer contacto que Hale había imaginado, pero ya\nera un principio. Le dirigió una mirada perpleja y regresó con parsimonia a su\nterminal. Mientras se sentaba, una cosa quedó muy clara: la encantadora Susan\nFletcher estaba trabajando en algo importante, y seguro que no se trataba de un\ndiagnóstico.'),
 Document(metadata={'source': 'data/fortaleza-digital.pdf', 'page': 2, 'page_label': '3', '_id': '64fd3506-ba62-4f1f-8dee-733f6d43321e', '_collection_name': 'db-book-page'}, page_content='Susan Fletcher, la criptógrafa estrella de la ultrasecreta Agencia de\nSeguridad Nacional (NSA) no puede dar crédito a sus oídos cuando su jefe,\nel subdirector de la Agencia, le informa de que han interceptado un código\nque ni siquiera la mayor supercomputadora conocida puede desc

In [10]:
to_rerank = []
for doc in docs:
    to_rerank.append((query, doc.page_content))

In [11]:
to_rerank

[('quién es susan fletcher?',
  'había tocado antes. No era el primer contacto que Hale había imaginado, pero ya\nera un principio. Le dirigió una mirada perpleja y regresó con parsimonia a su\nterminal. Mientras se sentaba, una cosa quedó muy clara: la encantadora Susan\nFletcher estaba trabajando en algo importante, y seguro que no se trataba de un\ndiagnóstico.'),
 ('quién es susan fletcher?',
  'Susan Fletcher, la criptógrafa estrella de la ultrasecreta Agencia de\nSeguridad Nacional (NSA) no puede dar crédito a sus oídos cuando su jefe,\nel subdirector de la Agencia, le informa de que han interceptado un código\nque ni siquiera la mayor supercomputadora conocida puede descifrar. La\núnica pista para romper el letal código parece estar oculta en el cadáver de\nun hombre que ha fallecido en España, donde ha sido enviado David, el\nprometido de Susan. Mientras éste intenta hallar la clave y sobrevivir a la\npersecución de un metódico e implacable asesino a sueldo en las calles de\nSe

In [12]:
output = reranker.compute_score(to_rerank, normalize=True)

In [13]:
output

[0.43194357648165793,
 0.982889110788156,
 0.929823701560032,
 0.9054895755643501,
 0.9014011355313043]

# Let's use cohere

In [24]:
import cohere
import os


co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

texts = [doc.page_content for doc in docs]

results = co.rerank(model="rerank-multilingual-v3.0", query=query, documents=texts, top_n=5)

In [25]:
results.results

[RerankResponseResultsItem(document=None, index=1, relevance_score=0.99987566),
 RerankResponseResultsItem(document=None, index=2, relevance_score=0.99647564),
 RerankResponseResultsItem(document=None, index=3, relevance_score=0.9927672),
 RerankResponseResultsItem(document=None, index=0, relevance_score=0.94407386),
 RerankResponseResultsItem(document=None, index=4, relevance_score=0.9339146)]

In [23]:
for t in texts:
    print(t)
    print("--"*10)

había tocado antes. No era el primer contacto que Hale había imaginado, pero ya
era un principio. Le dirigió una mirada perpleja y regresó con parsimonia a su
terminal. Mientras se sentaba, una cosa quedó muy clara: la encantadora Susan
Fletcher estaba trabajando en algo importante, y seguro que no se trataba de un
diagnóstico.
--------------------
Susan Fletcher, la criptógrafa estrella de la ultrasecreta Agencia de
Seguridad Nacional (NSA) no puede dar crédito a sus oídos cuando su jefe,
el subdirector de la Agencia, le informa de que han interceptado un código
que ni siquiera la mayor supercomputadora conocida puede descifrar. La
única pista para romper el letal código parece estar oculta en el cadáver de
un hombre que ha fallecido en España, donde ha sido enviado David, el
prometido de Susan. Mientras éste intenta hallar la clave y sobrevivir a la
persecución de un metódico e implacable asesino a sueldo en las calles de
Sevilla, Susan se enfrentará a su propio drama en las instalac